In [1]:
from keras.layers import Input, Dropout, Dense, Embedding
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
import pickle as pkl 
from sklearn.metrics import f1_score, classification_report
from layers.graph import SpectralGraphConvolution
from utils import *

Using TensorFlow backend.
/home/t/anaconda3/envs/gcn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/anaconda3/envs/gcn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/anaconda3/envs/gcn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/anaconda3/envs/gcn/lib/python3.6/sit

In [2]:
def fix_labels(labels):
    for i in range(len(labels)):
        if labels[i][0] == "I":
            if i == 0 or labels[i-1][2:] != labels[i][2:]:
                labels[i] = "B-{}".format(labels[i][2:])
    return labels


def decode_labels(labels, idx2label):
    labels = np.array(labels)
    prediction_indices = labels.argmax(axis=1)
    prediction_labels = [idx2label[i] for i in prediction_indices]
    return prediction_labels


def predict_labels(predictions, actuals, idx2label):
    predictions_labels = []
    actuals_labels = []
    for i in range(len(predictions)):
        prediction = predictions[i]
        actual = actuals[i]
        prediction_labels = decode_labels(prediction, idx2label)
        prediction_labels = fix_labels(prediction_labels)
        actual_labels = decode_labels(actual, idx2label)
        predictions_labels.append(prediction_labels)
        actuals_labels.append(actual_labels)
    return predictions_labels, actuals_labels

In [3]:
DATASET = 'conll2003'
EPOCHS = 4
LR = 0.001
L2 = 0
DO = 0.5
BATCH_SIZE = 8

In [4]:
print("Loading dataset...")

A, X, Y, meta = pkl.load(open('pkl/' + DATASET + '.pkl', 'rb'))

print("Loading embedding matrix...")

embedding_matrix = pkl.load(
    open('pkl/' + DATASET + '.embedding_matrix.pkl', 'rb'))

print("Processing dataset...")

val_y = load_output(A, X, Y, 'val')
test_y = load_output(A, X, Y, 'test')

num_nodes = A['train'][0][0].shape[0]
num_relations = len(A['train'][0]) - 1
num_labels = len(meta['label2idx'])

print("Number of nodes: {}".format(num_nodes))
print("Number of relations: {}".format(num_relations))
print("Number of classes: {}".format(num_labels))

Loading dataset...
Loading embedding matrix...
Processing dataset...
Number of nodes: 124
Number of relations: 44
Number of classes: 8


In [5]:
# Define model inputs
X_in = Input(shape=(num_nodes, ))
A_in = [Input(shape=(num_nodes, num_nodes)) for _ in range(num_relations)]

In [6]:
print("Define model")
# Define model architecture
X_embedding = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[
                        embedding_matrix], trainable=False)(X_in)
H = SpectralGraphConvolution(256, activation='relu')([X_embedding] + A_in)
H = Dropout(DO)(H)
H = SpectralGraphConvolution(256, activation='relu')([H] + A_in)
H = Dropout(DO)(H)
output = Dense(num_labels, activation='softmax')(H)

# Compile model
model = Model(inputs=[X_in] + A_in, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=LR))
model.summary()

Define model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 124)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 124, 300)     442806300   input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 124, 124)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 124, 124)     0                                            
________________________________________________________________________________________________

In [7]:
EPOCHS = 4
for epoch in range(EPOCHS):

    print("=== EPOCH {} ===".format(epoch + 1))

    model.fit_generator(batch_generator(A, X, Y, 'train', batch_size=BATCH_SIZE),
                        steps_per_epoch=len(A['train'])//BATCH_SIZE, verbose=1)

    val_predictions = model.predict_generator(batch_generator(
        A, X, Y, 'val', batch_size=BATCH_SIZE), steps=len(A['val'])//BATCH_SIZE, verbose=1)
    val_predicted_labels, val_actual_labels = predict_labels(
        val_predictions, val_y, meta['idx2label'])

    for i in range(len(val_predicted_labels)):
        val_predicted_labels[i] = [x.split('-')[1] if '-' in x else x for x in val_predicted_labels[i]]
    for i in range(len(val_actual_labels)):
        val_actual_labels[i] = [x.split('-')[1] if '-' in x else x for x in val_actual_labels[i]]
    
    gt = []
    pr = []
    for i in range(len(val_predicted_labels)):
        gt.extend(val_predicted_labels[i])
    for i in range(len(val_actual_labels)):
        pr.extend(val_actual_labels[i])
        
    print("=== Validation Results ===")
    print("Weighted F1-score: ",f1_score(gt,pr, average = 'weighted'))
    print("Classification report:\n", classification_report(gt,pr))

    test_predictions = model.predict_generator(batch_generator(
        A, X, Y, 'test', batch_size=8), steps=len(A['test']) // 8, verbose=1)

    test_predicted_labels, test_actual_labels = predict_labels(
        test_predictions, test_y, meta['idx2label'])
    for i in range(len(test_predicted_labels)):
        test_predicted_labels[i] = [x.split('-')[1] if '-' in x else x for x in test_predicted_labels[i]]
    for i in range(len(test_actual_labels)):
        test_actual_labels[i] = [x.split('-')[1] if '-' in x else x for x in test_actual_labels[i]]

    print("=== Test Results ===")

    gt = []
    pr = []
    for i in range(len(test_predicted_labels)):
        gt.extend(test_predicted_labels[i])
    for i in range(len(test_actual_labels)):
        pr.extend(test_actual_labels[i])
    print("Weighted F1-score: ",f1_score(gt,pr, average = 'weighted'))
    print("Classification report:\n", classification_report(gt,pr))


=== EPOCH 1 ===
Epoch 1/1
433/433 [==============================] - 105s 243ms/step
=== Validation Results ===
Weighted F1-score:  0.9972650997982003
Classification report:
               precision    recall  f1-score   support

         LOC       0.96      0.88      0.92      2272
        MISC       0.74      0.90      0.81      1047
           O       1.00      1.00      1.00    421201
         ORG       0.82      0.88      0.85      1934
         PER       0.95      0.97      0.96      3082

    accuracy                           1.00    429536
   macro avg       0.89      0.93      0.91    429536
weighted avg       1.00      1.00      1.00    429536

460/460 [==============================] - 111s 242ms/step
=== Test Results ===
=== Validation Results ===
Weighted F1-score:  0.9959189320097115
Classification report:
               precision    recall  f1-score   support

         LOC       0.91      0.79      0.84      2214
        MISC       0.67      0.76      0.71       802
   

In [8]:
# gt = []
# pr = []
# for i in range(len(test_predicted_labels)):
#     gt.extend(test_predicted_labels[i])
# for i in range(len(test_actual_labels)):
#     pr.extend(test_actual_labels[i])
# print("Weighted F1-score: ",f1_score(gt,pr, average = 'weighted'))
# print("Classification report:\n", classification_report(gt,pr))

In [9]:

# for i in range(len(val_predicted_labels)):
#     val_predicted_labels[i] = [x.split('-')[1] if '-' in x else x for x in val_predicted_labels[i]]
# for i in range(len(val_actual_labels)):
#     val_actual_labels[i] = [x.split('-')[1] if '-' in x else x for x in val_actual_labels[i]]
# gt = []
# pr = []
# for i in range(len(val_predicted_labels)):
#     gt.extend(val_predicted_labels[i])
# for i in range(len(val_actual_labels)):
#     pr.extend(val_actual_labels[i])
# print("Weighted F1-score: ",f1_score(gt,pr, average = 'weighted'))
# print("Classification report:\n", classification_report(gt,pr))

In [10]:
# f1_score(test_predicted_labels[10],test_actual_labels[10], average = 'macro')

In [11]:
# val_predicted_labels[10]

In [12]:
# val_actual_labels[10]

In [13]:
# val_predicted_labels, val_actual_labels = evaluation.predict_labels(
#         val_predictions, val_y, meta['idx2label'])

In [14]:
# for i in range(len(val_actual_labels)):
#     val_actual_labels[i] = [x.split('-')[1] if '-' in x else x for x in val_actual_labels[i]]